In [1]:
import os
import s3fs
from pathlib import Path

from decoimpact.business.application import Application
from decoimpact.business.workflow.model_builder import ModelBuilder
from decoimpact.crosscutting.i_logger import ILogger
from decoimpact.crosscutting.logger_factory import LoggerFactory
from decoimpact.data.entities.data_access_layer import DataAccessLayer, IDataAccessLayer

In [2]:
# Create an file-system object for the s3 bucket
onxia_user_name = os.environ["GIT_USER_NAME"]
bucket_name = f"oidc-{onxia_user_name}"
S3_ENDPOINT_URL = os.environ["S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

yaml_file = "input_file.yaml"
yaml_file_path = Path(yaml_file)

In [3]:
# download the yaml file
fs.download(f"{bucket_name}/{yaml_file}", yaml_file)

In [4]:
# configure logger and data-access layer for d-ecoimpact
logger: ILogger = LoggerFactory.create_logger()
da_layer: IDataAccessLayer = DataAccessLayer(logger)
model_builder = ModelBuilder(da_layer, logger)

In [5]:
# read the input (nc) and output (nc) path from the yaml input file
model_data = da_layer.read_input_file(yaml_file_path)
nc_file = str(model_data.datasets[0].path.relative_to(os.getcwd()))
output_file_path = str(model_data.output_path)

2024-01-10 16:34:56,625: INFO     Creating model data based on yaml file input_file.yaml


In [6]:
# download input netCDF (UGrid) file
fs.download(f"{bucket_name}/{nc_file}", nc_file)

In [7]:
# create and run d-ecoimpact application
application = Application(logger, da_layer, model_builder)
application.run(yaml_file_path)

2024-01-10 16:35:40,126: INFO     Application version: 0.1.11
2024-01-10 16:35:40,127: INFO     Creating model data based on yaml file input_file.yaml
2024-01-10 16:35:40,129: INFO     Input file version: 0.0.0
2024-01-10 16:35:40,130: INFO     Creating rule-based model
2024-01-10 16:35:40,154: INFO     Model "Model 1" -> ModelStatus.VALIDATING
2024-01-10 16:35:40,155: INFO     Model "Model 1" -> ModelStatus.VALIDATED
2024-01-10 16:35:40,155: INFO     Model "Model 1" -> ModelStatus.INITIALIZING
2024-01-10 16:35:40,157: INFO     Model "Model 1" -> ModelStatus.INITIALIZED
2024-01-10 16:35:40,157: INFO     Model "Model 1" -> ModelStatus.EXECUTING
2024-01-10 16:35:40,157: INFO     Starting rule test name
2024-01-10 16:35:40,163: INFO     Model "Model 1" -> ModelStatus.EXECUTED
2024-01-10 16:35:40,163: INFO     Model "Model 1" -> ModelStatus.FINALIZING
2024-01-10 16:35:40,163: INFO     Model "Model 1" -> ModelStatus.FINALIZED
2024-01-10 16:35:40,164: INFO     Model "Model 1" has successfull

In [8]:
# upload the output file to s3
fs.upload(output_file_path, f"{bucket_name}/{output_file_path}")